In [1]:
import os
import cv2
import numpy as np
from skimage import io, exposure
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
import csv

# Function to calculate GLCM features
def calculate_glcm_features(image):
    # Convert image to 8-bit (if necessary)
    image = exposure.rescale_intensity(image, out_range='uint8')

    # Calculate GLCM
    glcm = cv2.calcHist([image], [0], None, [256], [0, 256])
    glcm = cv2.normalize(glcm, glcm).flatten()

    # Calculate homogeneity, dissimilarity, correlation, and coarseness
    homogeneity = np.sum(glcm / (1 + np.arange(0, glcm.shape[0]))[:, None, None])
    
    # Correct the dissimilarity calculation
    dissimilarity = np.sum(np.abs(np.arange(0, glcm.shape[0])[:, None, None] - np.arange(0, glcm.shape[0])[None, None, :]) * glcm)
    
    correlation = np.sum(((np.arange(0, glcm.shape[0]) - np.mean(glcm)) / np.std(glcm))[:, None, None] * glcm)
    coarseness = np.sum(glcm * (np.arange(0, glcm.shape[0]) - np.arange(0, glcm.shape[0]))[:, None, None])

    return homogeneity, dissimilarity, correlation, coarseness

# Path to the folder containing images
folder_path = r"C:\Users\krithivarsha\.spyder-py3\Brain Tumor"

# Output CSV file path
csv_file_path = r"C:\Users\krithivarsha\.spyder-py3\4.csv"

# Create or overwrite the CSV file
with open(csv_file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    
    # Write header
    header = ['Mean', 'Variance', 'Standard Deviation', 'Entropy', 'Kurtosis', 'Skewness', 'Contrast', 'Energy', 'ASM',
              'Homogeneity', 'Dissimilarity', 'Correlation', 'Coarseness', 'PSNR', 'SSIM', 'MSE', 'DC']
    csv_writer.writerow(header)
    
    # Loop through all images in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            # Load the image
            image_path = os.path.join(folder_path, filename)
            image = io.imread(image_path, as_gray=True)
            
            # Calculate GLCM features
            homogeneity_value, dissimilarity_value, correlation_value, coarseness_value = calculate_glcm_features(image)
            
            # Calculate mean, variance, standard deviation, entropy, kurtosis, skewness, contrast, energy, ASM
            mean_value = np.mean(image)
            variance_value = np.var(image)
            std_deviation_value = np.std(image)
            entropy_value = -np.sum(image * np.log2(image + 1e-10))
            kurtosis_value = np.mean(((image - mean_value) / np.std(image))**4) - 3
            skewness_value = np.mean(((image - mean_value) / np.std(image))**3)
            contrast_value = np.mean((image - mean_value)**2)
            energy_value = np.sum(image**2)
            asm_value = energy_value / image.size
            
            # Calculate PSNR, SSIM, MSE, and DC
            original_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image, (original_image.shape[1], original_image.shape[0]))
            psnr_value = peak_signal_noise_ratio(original_image, image, data_range=255)
            ssim_value, _ = structural_similarity(original_image, image, full=True, data_range=255)
            mse_value = np.mean((original_image - image)**2)
            dc_value = np.sum(np.minimum(original_image, image)) / np.sum(original_image)
            
            # Write the values to the CSV file
            row = [mean_value, variance_value, std_deviation_value, entropy_value, kurtosis_value, skewness_value,
                   contrast_value, energy_value, asm_value, homogeneity_value, dissimilarity_value, correlation_value,
                   coarseness_value, psnr_value, ssim_value, mse_value, dc_value]
            csv_writer.writerow(row)

print(f"Features saved to {csv_file_path}")

C:\Users\krithivarsha\AppData\Roaming\Python\Python311\site-packages\skimage\metrics\simple_metrics.py:163: RuntimeWarning: divide by zero encountered in scalar divide
  return 10 * np.log10((data_range ** 2) / err)


KeyboardInterrupt: 